In [1]:
# Scrape data

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import pandas as pd
import time
from pathlib import Path
import os

In [2]:
def init_driver(headless=True):

    # if headless:
    #     options = webdriver.ChromeOptions()
    #     # Run in headless mode (No GUI needed)
    #     options.add_argument("--headless")
    #     options.add_argument("--no-sandbox")
    #     options.add_argument("--disable-dev-shm-usage")
    #     options.add_argument("--disable-gpu")  # Required for WSL2
    #     options.add_argument("--remote-debugging-port=9222")  # Debugging support

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    return driver

def navigate_to_page(driver, url):
    """Navigate to the specified URL."""
    driver.get(url)

def select_area(driver, wait, area_name):
    """Select the area by name."""
    try:
        area_selector = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "select2-search__field")))
        area_selector.click()
        area_selector.send_keys(area_name)
        time.sleep(1) # Set a delay to wait a key be filled in the input field
        area_selector.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"Error selecting area '{area_name}': {e}")

def input_date(driver, wait, xpath, date_value):
    """Input a date into the specified field."""
    try:
        date_field = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        # Clear the field before entering the new value  
        date_field.send_keys(Keys.CONTROL + "a")  
        date_field.send_keys(Keys.BACKSPACE)
        date_field.send_keys(date_value)  # Enter the new value
    except Exception as e:
        print(f"Error inputting date '{date_value}': {e}")

def select_time(driver, wait, xpath, time_value):
    """Select a time from the dropdown."""
    try:
        time_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        time_dropdown.click()
        time_dropdown.send_keys(time_value)
        time_dropdown.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"Error selecting time '{time_value}': {e}")

def click_radio_button(driver, xpath):
    """Click a radio button using its visible text."""
    try:
        radio_span = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        radio_span.click()
    except Exception as e:
        print(f"Error clicking radio button: {e}")

def click_button(driver, xpath):
    """Click a button using its XPath."""
    try:
        button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        button.click()
    except Exception as e:
        print(f"Error clicking button: {e}")

def extract_table(driver, xpath, delay=10):
    """
    Extracts data from a table identified by its XPath with a delay to ensure the table is loaded.
    
    Args:
        driver: Selenium WebDriver instance.
        xpath: XPath to locate the table.
        delay: Time in seconds to wait for the table to load.
    
    Returns:
        pd.DataFrame: A DataFrame containing the table data.
    """
    try:
        # Wait for the table and locate it
        table = WebDriverWait(driver, delay).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        
        # Locate table header
        header = table.find_element(By.XPATH, ".//thead/tr")
        time.sleep(0.5)
        header_cells = header.find_elements(By.TAG_NAME, "th")
        #header_data = [cell.text.strip() for cell in header_cells]
        header_data = []
        for cell in header_cells:
            header_data.append(cell.text.strip())
        
        # Locate all rows in the tbody
        table_data = []
        rows = table.find_elements(By.XPATH, ".//tbody/tr")
        for row in rows:
            # Extract all cells from the row
            cells = row.find_elements(By.TAG_NAME, "td")
            row_data = [cell.text.strip() for cell in cells]
            table_data.append(row_data)
        
        # Convert to DataFrame
        return pd.DataFrame(table_data, columns=header_data)
    
    except Exception as e:
        print(f"Error extracting data from the table: {e}")
        return pd.DataFrame()
    
# def scrape_data(driver, wait, url, area_name, start_date, end_date, start_time, end_time):
#     """Scrape data from the specified URL."""
#     # Navigate to the URL
#     navigate_to_page(driver, url)
    
#     # Select the area
#     select_area(driver, wait, area_name)
    
#     # Input the start date
#     input_date(driver, wait, "//div[@id='startdate']//input[@type='text']", start_date)

#     # Select the start time
#     select_time(driver, wait, "//select[@value.bind='TimeStart']", start_time)

#     # Input the end date
#     input_date(driver, wait, "//div[@id='enddate']//input[@type='text']", end_date)
    
#     # Select the end time
#     select_time(driver, wait, "//select[@value.bind='TimeEnd']", end_time)
    
#     # Click the radio button for the table view
#     click_radio_button(driver, "//span[text()='ตาราง']")
    
#     # Click the 'View Data' button
#     click_button(driver, "//button[@click.delegate='SearchFunction()']")
    
#     # Extract the table data
#     table_xpath = "//table[@id='table' and contains(@class, 'table-gradient table-striped')]"
#     return extract_table(driver, table_xpath)

def is_not_empty(csv_path):
    """Check whether CSV is empty"""
    try:
        df = pd.read_csv(csv_path)
        if not df.empty:
            return df, True
        else:
            print(f"CSV file '{csv_path}' is empty.")
            return None, False
    except Exception as e:
        print(f"Error reading CSV file '{csv_path}': {e}")
        return None, False
    
def is_column_correct(df, expected_columns):
    """Validate CSV file columns against expected columns"""
    if all(x == y for x, y in zip(df.columns, expected_columns)):
        return True
    else:
        print(f"CSV file has incorrect columns")
        return False
    
# def is_column_correct(csv_path, expected_columns):
#     """Validate CSV file columns against expected columns"""
#     try:
#         df = pd.read_csv(csv_path)
#         if all(x == y for x, y in zip(df.columns, expected_columns)):
#             return True
#         else:
#             print(f"CSV file '{csv_path}' has incorrect columns")
#             return False
#     except Exception as e:
#         print(f"Error reading CSV file '{csv_path}': {e}")
#         return False
    
def is_24hr_records(df):
    """Check whether CSV file contains 24 rows of record"""
    if len(df) == 24:
        return True
    else:
        print(f"CSV file has data less than 24 hours of record")
        return False
        
    
def validate_csv(csv_path, expected_columns):
    """Validate the CSV file"""

    df, not_empty = is_not_empty(csv_path)

    if not_empty:
        if is_column_correct(df,expected_columns) and is_24hr_records(df):
            return True
    else:
        return False

        
    # # Check csv is empty
    # if is_not_empty(csv_path):
    #     # Check column name
    #     if is_column_correct(csv_path, expected_columns):
    #         return True
    #     else:
    #         return False
    # else:
    #     return False


In [3]:
district = "เขตวัฒนา"
dest_root = "../data"

driver = init_driver()
url = "https://airquality.airbkk.com/PublicWebClient/#/Modules/Aqs/DashboardPage"
navigate_to_page(driver, url)
    
wait = WebDriverWait(driver, 20)

start_date_str = "01/03/2023"
end_date_str = "31/12/2024"
start_date = datetime.strptime(start_date_str, "%d/%m/%Y")
end_date = datetime.strptime(end_date_str, "%d/%m/%Y")

# Loop on district and date, incrementing by 1 day
for i in range((end_date - start_date).days + 1):

    current_date = start_date + timedelta(days=i)
    current_date_str = current_date.strftime("%d/%m/%Y")

    # Step 1: Select area
    if i == 0:
        select_area(driver, wait, f"{district}")

    # Step 2: Input start date
    input_date(driver, wait, "//div[@id='startdate']//input[@type='text']", current_date_str)

    # Step 3: Select start time
    select_time(driver, wait, "//select[@value.bind='TimeStart']", "00:00")

    # Step 4: Input end date
    input_date(driver, wait, "//div[@id='enddate']//input[@type='text']", current_date_str)

    # Step 5: Select end time
    select_time(driver, wait, "//select[@value.bind='TimeEnd']", "23:00")

    # Step 6: Click the radio button
    click_button(driver, "//span[text()='ตาราง']")

    # Step 7: Click the search button
    click_button(driver, "//button[@click.delegate='SearchFunction()']")

    # Step 8: Extract the table data
    data_df = extract_table(driver, "//table[@id='table' and contains(@class, 'table-gradient table-striped')]")

    # Save df to csv
    current_date_str_re = current_date.strftime("%Y-%m-%d") 
    year, month, day = current_date_str_re.split("-")
    os.makedirs(os.path.join(dest_root, district, year, month), exist_ok=True)
    file_path = Path(os.path.join(dest_root, f"{district}/{year}/{month}/{current_date_str_re}.csv"))
    data_df.to_csv(file_path, index=False)

    print(f"Scraping data on {current_date_str} --> save to {file_path}")

driver.quit()

Error extracting data from the table: 13 columns passed, passed data had 1 columns
Scraping data on 01/03/2023 --> save to ../data/เขตวัฒนา/2023/03/2023-03-01.csv
Error extracting data from the table: 13 columns passed, passed data had 1 columns
Scraping data on 02/03/2023 --> save to ../data/เขตวัฒนา/2023/03/2023-03-02.csv
Error extracting data from the table: 13 columns passed, passed data had 1 columns
Scraping data on 03/03/2023 --> save to ../data/เขตวัฒนา/2023/03/2023-03-03.csv


KeyboardInterrupt: 

In [12]:
district

'เขตคลองเตย'

In [13]:
# Validate the CSV files

columns = ['เขต', 'สถานีตรวจวัด', 'วันที่และเวลา', 'PM10', 'PM2.5', 'CO', 'NO2',
            'O3', 'Wind Speed', 'Wind Direction', 'Temperature',
            'Relative Humidity', 'Air Pressure']

driver = init_driver()
url = "https://airquality.airbkk.com/PublicWebClient/#/Modules/Aqs/DashboardPage"
navigate_to_page(driver, url)
wait = WebDriverWait(driver, 20)

select_area(driver, wait, district)

dest_root = "../data"
folder_path = Path(os.path.join(dest_root, district))
for fpath in os.walk(folder_path):
    for file in fpath[2]:
        file_path = Path(os.path.join(fpath[0], file))
        print(file_path)

        if not validate_csv(file_path, expected_columns=columns):
            print(f"Re-scraping data for {file_path}")

            # Extract the date from the file name
            year, month, day = file_path.stem.split("-")
            current_date = datetime(int(year), int(month), int(day))
            current_date_str = current_date.strftime("%d/%m/%Y")

            # Re-scrape the data
            # Step 2: Input start date
            input_date(driver, wait, "//div[@id='startdate']//input[@type='text']", current_date_str)

            # Step 3: Select start time
            select_time(driver, wait, "//select[@value.bind='TimeStart']", "00:00")

            # Step 4: Input end date
            input_date(driver, wait, "//div[@id='enddate']//input[@type='text']", current_date_str)

            # Step 5: Select end time
            select_time(driver, wait, "//select[@value.bind='TimeEnd']", "23:00")

            # Step 6: Click the radio button
            click_button(driver, "//span[text()='ตาราง']")

            # Step 7: Click the search button
            click_button(driver, "//button[@click.delegate='SearchFunction()']")

            # Step 8: Extract the table data
            data_df = extract_table(driver, "//table[@id='table' and contains(@class, 'table-gradient table-striped')]")

            # Save df to csv
            current_date_str_re = current_date.strftime("%Y-%m-%d") 
            year, month, day = current_date_str_re.split("-")
            os.makedirs(os.path.join(dest_root, district, year, month), exist_ok=True)
            file_path = Path(os.path.join(dest_root, f"{district}/{year}/{month}/{current_date_str_re}.csv"))
            data_df.to_csv(file_path, index=False)

            print(f"Scraping data on {current_date_str} --> save to {file_path}")
            print(f"Finish re-scraping data for {file_path}")

driver.quit()

../data/เขตคลองเตย/2023/03/2023-03-01.csv
../data/เขตคลองเตย/2023/03/2023-03-02.csv
../data/เขตคลองเตย/2023/03/2023-03-03.csv
../data/เขตคลองเตย/2023/03/2023-03-04.csv
../data/เขตคลองเตย/2023/03/2023-03-05.csv
CSV file has data less than 24 hours of record
Re-scraping data for ../data/เขตคลองเตย/2023/03/2023-03-05.csv
Scraping data on 05/03/2023 --> save to ../data/เขตคลองเตย/2023/03/2023-03-05.csv
Finish re-scraping data for ../data/เขตคลองเตย/2023/03/2023-03-05.csv
../data/เขตคลองเตย/2023/03/2023-03-06.csv
../data/เขตคลองเตย/2023/03/2023-03-07.csv
../data/เขตคลองเตย/2023/03/2023-03-08.csv
../data/เขตคลองเตย/2023/03/2023-03-09.csv
../data/เขตคลองเตย/2023/03/2023-03-10.csv
../data/เขตคลองเตย/2023/03/2023-03-11.csv
../data/เขตคลองเตย/2023/03/2023-03-12.csv
../data/เขตคลองเตย/2023/03/2023-03-13.csv
../data/เขตคลองเตย/2023/03/2023-03-14.csv
../data/เขตคลองเตย/2023/03/2023-03-15.csv
../data/เขตคลองเตย/2023/03/2023-03-16.csv
../data/เขตคลองเตย/2023/03/2023-03-17.csv
../data/เขตคลองเตย/2023/